### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра информационных технологий 

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2


### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Бармина Ольга Константиновна
##### Группа:   НФИбд-01-19

## Москва 2022
***

### Вариант №25

задание:

1.	При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

2.  Загрузите таблицы `departments` и `products` в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы `order_products__train` записи, соответствующие указанным в индивидуальном задании дню недели (поле `order_dow` таблицы `orders`) и коду департамента (поле `department_id` таблицы `products`) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме и определите количество товаров (стоолбец `product_id`) в транзакциях датафрейма.

3.	Определите пять наиболее популярных товаров в датафрейме транзакций и определите количество покупок (транзакций) этих товаров.

4.	Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец `order_id`, а в качестве названий товаров - поле `product_name` из датафрейма для таблицы `products`, соответствующее столбцу `product_id`. Найдите в транзакционной базе данных три транзакции с наибольшим количеством товаров и выведите их на экран. 

5.	Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета `mlxtend`. По бинарной базе данных определите пять  наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.

6.	При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

7.	Используя пакет `mlxtend` или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов. Используйте уровень достоверности (confidence), равный 0.6.

8.	Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки.


Индивидуальный вариант: 

1. Алгоритм: FPGrowth 

2. День недели (поле order_dow таблицы orders): “3” 

3. Код департамента (поле department_id таблицы products): “11” 

4. Показатель оценки ассоциативных правил: лифт (lift) 

In [1]:
import numpy as np
import pandas as pd
import sqlite3

# откроем базу данных
conn = sqlite3.connect('instacart.db')
cursor = conn.cursor()

In [2]:
# создадим датафреймя из таблиц departments и orders
df_dept = pd.read_sql_query("SELECT * FROM departments", conn)
df_dept = df_dept.set_index('department_id')
df_dept

,department
department_id,
1,frozen
2,other
3,bakery
4,produce
5,alcohol
6,international
7,beverages
8,pets
9,dry goods pasta


In [3]:
df_prod = pd.read_sql_query("SELECT * FROM products", conn)
df_prod

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [4]:
# select p.product_name as prod

# посмотрим таблицу из записей, удовлетворяющих условиям варианта
for row in cursor.execute("""
    select opp.order_id, opp.product_id, opp.add_to_cart_order, opp.reordered
    from order_products__train opp, products p, orders ord
    where p.product_id = opp.product_id 
    AND ord.order_id = opp.order_id 
    AND ord.order_dow = '3' 
    AND p.department_id = '11'
    """):
    print(row)

('1472290', '22', '1', '0')
('257077', '146', '2', '1')
('372892', '146', '2', '0')
('237826', '203', '23', '1')
('218041', '251', '5', '0')
('1263923', '280', '3', '1')
('1516183', '280', '1', '1')
('2132926', '280', '3', '0')
('2293421', '280', '3', '0')
('231694', '280', '4', '0')
('2634727', '280', '1', '1')
('2861288', '280', '2', '1')
('2964680', '280', '2', '1')
('3046057', '280', '14', '0')
('750908', '280', '3', '1')
('850313', '280', '25', '0')
('2082667', '308', '14', '1')
('1310415', '351', '12', '0')
('1445427', '351', '2', '0')
('2517054', '351', '13', '1')
('3345484', '351', '9', '0')
('2002001', '361', '2', '0')
('2821696', '366', '7', '0')
('409962', '394', '6', '0')
('2507314', '401', '7', '0')
('3332494', '401', '8', '1')
('44784', '401', '2', '0')
('1406365', '408', '2', '1')
('321294', '455', '10', '0')
('3251301', '455', '1', '0')
('5459', '455', '31', '0')
('809763', '455', '1', '0')
('3389913', '498', '5', '1')
('2446435', '563', '10', '0')
('2571006', '603', '5

('2836426', '15658', '19', '0')
('109327', '15685', '5', '0')
('1982996', '15746', '11', '0')
('2658118', '15746', '6', '0')
('3135206', '15746', '4', '0')
('1474548', '15760', '7', '0')
('1371439', '15861', '5', '0')
('1110260', '15887', '21', '0')
('1518310', '15887', '18', '1')
('255087', '15887', '30', '0')
('2957899', '15887', '7', '0')
('3362099', '15887', '8', '0')
('888633', '15887', '6', '0')
('100962', '15898', '15', '0')
('1306514', '15904', '12', '0')
('1923345', '15904', '4', '0')
('1084548', '15965', '18', '0')
('935674', '16000', '9', '1')
('2814772', '16019', '16', '0')
('2865254', '16019', '1', '0')
('929875', '16019', '7', '0')
('2608764', '16042', '2', '0')
('276655', '16042', '7', '0')
('33680', '16042', '10', '0')
('3417483', '16042', '28', '0')
('2365792', '16082', '4', '1')
('1745375', '16122', '2', '1')
('3275148', '16297', '2', '1')
('2398750', '16324', '1', '1')
('1191415', '16327', '8', '0')
('3012208', '16367', '1', '0')
('1241026', '16393', '23', '0')
('187

('937531', '34521', '9', '0')
('2146786', '34524', '5', '0')
('2155203', '34536', '1', '0')
('567439', '34614', '13', '0')
('661323', '34614', '5', '0')
('806060', '34659', '11', '1')
('116188', '34699', '4', '0')
('238854', '34699', '42', '0')
('842657', '34699', '3', '0')
('374863', '34799', '2', '1')
('2498168', '34819', '5', '1')
('2516281', '34819', '20', '1')
('54653', '34819', '10', '1')
('1295633', '34845', '6', '1')
('347550', '34859', '14', '0')
('197679', '34894', '5', '0')
('2709735', '34894', '5', '1')
('3015051', '34894', '1', '1')
('174624', '34912', '8', '0')
('2475239', '34924', '2', '0')
('1983710', '34964', '4', '0')
('2259700', '35018', '5', '0')
('2349027', '35018', '10', '0')
('2571006', '35018', '6', '0')
('983282', '35018', '2', '0')
('3305645', '35039', '1', '0')
('165674', '35134', '1', '0')
('2835212', '35134', '4', '1')
('3379094', '35134', '4', '1')
('198845', '35165', '1', '0')
('2221284', '35165', '28', '0')
('298585', '35165', '11', '0')
('610582', '3516

In [5]:
# запишем эту таблицу в датафрейм
df_opp = pd.read_sql_query("""
                        select opp.order_id, opp.product_id, p.product_name, opp.add_to_cart_order, opp.reordered
                        from order_products__train opp, products p, orders ord
                        where p.product_id = opp.product_id 
                        AND ord.order_id = opp.order_id 
                        AND ord.order_dow = '3' 
                        AND p.department_id = '11'""", conn)
df_opp
# получаем, что нашим условия удовлетворяют 2556 записей

,order_id,product_id,product_name,add_to_cart_order,reordered
0,1472290,22,Fresh Breath Oral Rinse Mild Mint,1,0
1,257077,146,Anti Diarrheal Caplets,2,1
2,372892,146,Anti Diarrheal Caplets,2,0
3,237826,203,"Rescue Remedy, Spray",23,1
4,218041,251,Extra Strength Melatonin Adult Gummies,5,0
...,...,...,...,...,...
2551,1793846,49433,3D White Whitestrips Luxe Glamorous White,20,0
2552,2565422,49501,Pro Health Antigingivitis And Sensitive Teeth ...,21,1
2553,2823643,49572,Honey & Lemon Menthol Cough Suppressants,13,0
2554,1476570,49649,Refreshing Remover Cleansing Towelettes,5,1


In [70]:
# посмотрим количество уникальных продуктов
np.size(df_opp['product_name'].unique())

1421

In [6]:
# посмотрим 5 самых часто заказываемых товаров
df5 = df_opp.groupby('product_name').count().nlargest(5, 'order_id')['order_id']
df5

product_name
Lavender Hand Soap                                                      36
Cotton Swabs                                                            32
Fluoride-Free Antiplaque & Whitening Peppermint Toothpaste              18
Lemon Verbena Hand Soap                                                 18
Natural Anticavity Silly Strawberry Fluoride Toothpaste for Children    15
Name: order_id, dtype: int64

In [12]:
# для построения транзакционной БД объеденим таблицы products и order_products__train, оставив 
# только 2 стобца - номер заказа и название продукта
df_tmp = pd.merge(df_prod, df_opp, on='product_id')[['order_id','product_name_x']].copy()
df_tmp.columns = ['order_id', 'product_name']
df_tmp

,order_id,product_name
0,1472290,Fresh Breath Oral Rinse Mild Mint
1,257077,Anti Diarrheal Caplets
2,372892,Anti Diarrheal Caplets
3,237826,"Rescue Remedy, Spray"
4,218041,Extra Strength Melatonin Adult Gummies
...,...,...
2551,1793846,3D White Whitestrips Luxe Glamorous White
2552,2565422,Pro Health Antigingivitis And Sensitive Teeth ...
2553,2823643,Honey & Lemon Menthol Cough Suppressants
2554,1476570,Refreshing Remover Cleansing Towelettes


In [13]:
# преобразуем полученную таблицу в словарь, а затем пройдем по всем элементам словаря и занесем их в список
df_new = df_tmp.groupby('order_id')['product_name'].apply(list).to_dict().items()
df = []
for row in df_new:
    print(row)
    df.append(row)

('1000595', ['Cardamom SmartFloss'])
('1001169', ['Ibuprofen'])
('1004642', ['Ultra Strength Assorted Berries Antacid'])
('100772', ['Watermelon 3 in 1 Shampoo Conditioner and Body Wash', 'Original Clean Dry Spray Antiperspirant Deodorant', 'Deep Moisture Pump Body Wash', 'Oatmeal & Shea Butter Body Lotion'])
('1009556', ['FreshBurst Antiseptic Mouthwash'])
('100962', ['Mango Peach Omega Swirl Omage-3 Fish Oil Supplement'])
('1011502', ['Purifying Tea Tree Body Wash'])
('1013793', ['Age Defying Daily Facial Moisturizer'])
('1014185', ['Natural Herb Cough Drops'])
('1021451', ['French Lavender Body Lotion', 'Total Moisture Aloe Fresh Hydrating Lotion', 'Ultimate Healing Aloe Skin Therapy Lotion'])
('1021481', ['Gotu Kola Stem Cell + 1% CGF Day Cream'])
('1021685', ['Truly Radiant Whitening & Enamel Strengthening Fresh Mint Toothpaste'])
('1024072', ['Grapeseed Natural Skin Care Oil'])
('1026064', ['Hand Soap Lavender & Coconut', 'Tea Tree Oil Dental Floss', "Vanilla Al'mondo Protein Sha

In [14]:
# найдем 3 транзакции с наибольшим количеством товаров
# отсортируем список по убываюнию количества элементов во втором элементе подсписков
# выведем 3 первых элемента
df.sort(key = lambda x:np.size(x[1]), reverse=True)
df[0], df[1], df[2]

(('3108267',
  ['Cleansing Towelettes Night Calming Makeup Remover',
   'Plus Soft Toothbrush',
   'Pain Reliever and Fever Reducer Tablets',
   'Total Clean Mint Toothpaste',
   'Loofah Sponges',
   'Neosporin 24 Hour Infection Protection First Aid Antibiotic Ointment',
   'Cotton Swabs',
   'Serenity Bubble Bath',
   'Deep Cleansing Pore Strips']),
 ('1300033',
  ['Wicked Fresh! Cool Peppermint Toothpaste',
   'Natural Anticavity Silly Strawberry Fluoride Toothpaste for Children',
   'Coconut Milk Nourishing Shampoo',
   'Pro-Health Stages Kids Toothbrush With Minnie Mouse',
   'Disney Cars Soft Toothbrush Pro Health Stages 5-7 Yrs',
   'Classic Cherry Lipbalm',
   'Calming Lavender Body Wash',
   'Classic Mouthwash Original Mint Flavor']),
 ('2194307',
  ['Plus Antibiotic Brand Adhesive Bandages',
   'Hurt-Free Antiseptic Wash',
   'Tough Strips',
   'Absolute Waterproof Tape',
   'Nexcare Waterproof Assorted Bandages',
   'Hydrogen Peroxide',
   'Sheer Extra Large All One Size 1 3/

In [19]:
# для преобразования в бинарную БД избавимся от order_id
df_products = [x[1] for x in df]
df_products

[['Cleansing Towelettes Night Calming Makeup Remover',
  'Plus Soft Toothbrush',
  'Pain Reliever and Fever Reducer Tablets',
  'Total Clean Mint Toothpaste',
  'Loofah Sponges',
  'Neosporin 24 Hour Infection Protection First Aid Antibiotic Ointment',
  'Cotton Swabs',
  'Serenity Bubble Bath',
  'Deep Cleansing Pore Strips'],
 ['Wicked Fresh! Cool Peppermint Toothpaste',
  'Natural Anticavity Silly Strawberry Fluoride Toothpaste for Children',
  'Coconut Milk Nourishing Shampoo',
  'Pro-Health Stages Kids Toothbrush With Minnie Mouse',
  'Disney Cars Soft Toothbrush Pro Health Stages 5-7 Yrs',
  'Classic Cherry Lipbalm',
  'Calming Lavender Body Wash',
  'Classic Mouthwash Original Mint Flavor'],
 ['Plus Antibiotic Brand Adhesive Bandages',
  'Hurt-Free Antiseptic Wash',
  'Tough Strips',
  'Absolute Waterproof Tape',
  'Nexcare Waterproof Assorted Bandages',
  'Hydrogen Peroxide',
  'Sheer Extra Large All One Size 1 3/4\\" X 4\\" Value'],
 ['Rescue Remedy, Spray',
  'ColdCare, Sooth

In [20]:
# построим бинарную БД по транзакционной БД
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
dataset_bin = te.fit(df_products).transform(df_products)
df_bin = pd.DataFrame(dataset_bin, columns=te.columns_)
df_bin

,1 Mg Melatonin Sublingual Orange Tablets,1 Razor Handle and 2 Freesia Scented Razor Refills Premium BladeRazor System,"1% Hydrocortisone Anti-Itch Cream, Tube Anti-Itch","1,000 Mg Vitamin C Super Orange","1,000 mg Vitamin C Lemon-Lime Flavored Fizzy Drink Mix - 30 PK",100% Cotton Rounds,100% Cotton Swabs,100% Pure Sensitive Skin Care Grape Seed Oil,100% Whey Protein Vanilla Flavor,1000 Roses Cleansing Foam for Sensitive Skin,...,Xtra Protection Regular Pantiliners,Yerbamate Lemon Energy Shot,Yoga Bath Soak,Zinc Echinacea Cherry Lozenges,Zinc Elderberry and Raspberry Lozenges,ZuZu Luxe Onyx Mascara,ZzzQuil Liquid Warming Berry Flavor Sleep-Aid,for All Mouth Sores Fresh Mint Oral Debriding Agent/Antiseptic Rinse/Pain Reliever,gel hand wash sea minerals,go fresh Cool Moisture Beauty
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1783,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1784,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1785,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
# найдем 5 самых популярных товаров и количество их заказов
df_bin.sum(axis=0).nlargest(5)

Lavender Hand Soap                                                      36
Cotton Swabs                                                            32
Fluoride-Free Antiplaque & Whitening Peppermint Toothpaste              18
Lemon Verbena Hand Soap                                                 18
Natural Anticavity Silly Strawberry Fluoride Toothpaste for Children    15
dtype: int64

In [65]:
# построим минимальный набор предметов с поддержкой не менее 0.001
# (если брать больше, то списки предметов получаются только одноэлементные, что мешает выполнению след. пункта)
from mlxtend.frequent_patterns import fpgrowth

itemsets_fpg = fpgrowth(df_bin, min_support=0.001, use_colnames=True)
itemsets_fpg

,support,itemsets
0,0.017907,(Cotton Swabs)
1,0.003917,(Cleansing Towelettes Night Calming Makeup Rem...
2,0.002798,(Pain Reliever and Fever Reducer Tablets)
3,0.001679,(Deep Cleansing Pore Strips)
4,0.001679,(Serenity Bubble Bath)
...,...,...
475,0.001119,"(Cotton Rounds, Coconut Milk Nourishing Shampoo)"
476,0.001119,"(Lavender Hand Soap, Lemon Verbena Hand Soap)"
477,0.001119,"(Lavender Hand Soap, Olive Oil & Aloe Vera Han..."
478,0.001119,"(Spearmint + Lemongrass Hand Soap, Hand Soap L..."


In [66]:
# построим набор ассоциативных правил
from mlxtend.frequent_patterns import association_rules

rules = association_rules(itemsets_fpg, metric="confidence", min_threshold=0.6)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Moroccan Argan Oil + Argan Stem Cell Triple M...,(Hair Shampoos),0.001679,0.002798,0.001119,0.666667,238.266667,0.001114,2.991606
1,(Meyer Lemon Everyone Hand Soap),(Children's Outrageous Orange Mango Natural Fl...,0.001679,0.002798,0.001119,0.666667,238.266667,0.001114,2.991606
2,(Active Naturals Positively Nourishing Body Wa...,(Soothing Aloe Vera Moisturizing Body Wash),0.001119,0.002798,0.001119,1.000000,357.400000,0.001116,inf
3,(Spearmint + Lemongrass Hand Soap),(Hand Soap Lavender & Coconut),0.001119,0.004477,0.001119,1.000000,223.375000,0.001114,inf
4,(All One Hemp Peppermint Castile Soap Bar),(All One Hemp Lavender Castile Soap Bar),0.002238,0.004477,0.001679,0.750000,167.531250,0.001669,3.982093


In [68]:
# найдем 2 правила с лучшим показателем lift
rules.sort_values(['lift'], ascending=False).head(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Active Naturals Positively Nourishing Body Wa...,(Soothing Aloe Vera Moisturizing Body Wash),0.001119,0.002798,0.001119,1.000000,357.400000,0.001116,inf
0,(Moroccan Argan Oil + Argan Stem Cell Triple M...,(Hair Shampoos),0.001679,0.002798,0.001119,0.666667,238.266667,0.001114,2.991606
